# Spotify API features for unsupervised methods

### Goal: 

Create a dataset of audio attributes derived from the Spotify API for use with genre clustering and acoustical similarity methods 

### Import Libraries and create spotify credentials manager

In [1]:
#importing module spotipy to make api calls to spotify
import spotipy
# module to extract song from url
import urllib
# OAuth Credenials
from spotipy.oauth2 import SpotifyClientCredentials
from multiprocessing import Process, Queue
import pandas as pd 
import numpy as np
from sklearn import datasets, linear_model
#from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

client_credentials_manager = SpotifyClientCredentials(client_id='4a1a457d14c0445a96e7021a5523905b', client_secret='45b186d601544fdcbb4fe07c7b23cce5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


### Read in csv that has the track ids from all Spotify playlists

In [2]:
track_info = pd.read_csv("kmeans_output.csv")

In [3]:
track_ids = list(track_info['id'])

In [4]:
#import glob, os
#os.chdir("/Users/sonumehta/Desktop/Sem2/Capstone/PyAudio_tools/pyAudioAnalysis/tracks_total")
#track_ids  = []
#for file in glob.glob("*.wav"):
#    track_ids.append(file[5:-4])



In [5]:
len(track_ids)
print(track_ids[:15])

['1tPGSiwtvmMVAqmniJ7oHp', '1tpShM80HMT9vKjRiA3EzW', '1TPUbLdzpfgF7wrPEXIAAB', '1tQGRq2WOBXjL3JWdWMONg', '1TqmZVIyeD4aN4Wa0wbnfy', '1TquBahz02HCaxj8jy5tTo', '1Tr4K5MU5XYE44umXGDndd', '1tRlCCMVLSVQrrN9PPgtf0', '1TScJLsDWzDVBFpCU5vJDo', '1TSjS0q24cPW2S4pYj2wEK', '1Tt4sE4pXi57mTD1GCzsqm', '1TtdHZ4KvN56LCt9evBUaL', '1TTiZwoorWMR4KmaI0irjU', '1TtJJ7YP6JCQ3aH8PQpreD', '1TTOAw90zzNWKnUgzzcvZW']


In [6]:
track_result = sp.tracks(["1tpShM80HMT9vKjRiA3EzW"])

In [12]:
genres = {
  "genres" : [ "acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-dubstep", "power-pop", "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "rainy-day", "reggae", "reggaeton", "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo", "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish", "study", "summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", "work-out", "world-music" ]
}

In [13]:
genres['genres']

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [14]:
sp.artist(track_result['tracks'][0]['artists'][0]['id'])['genres']

['folk',
 'folk christmas',
 'folk-pop',
 'lilith',
 'mellow gold',
 'neo mellow',
 'new wave pop',
 'permanent wave',
 'pop christmas',
 'pop rock',
 'rock',
 'singer-songwriter']

### For each track, get the audio attributes and other Spotify API features

In [76]:
def get_popularity(numLoops, track_ids):
    # store all tracks of all playlists 
    print('Starting')
    track_ids_all= []
    popularity = []
    preview_urls_all =[]
    artists_all = []
    names_all = []
    track_numbers_all = []
    # set parameters
    offset=0
    limit=50
    aud_features = np.empty((limit*numLoops, 13))
    for loopIndex in range(numLoops):
        start = offset + limit*loopIndex # start 
        tracks = track_ids[start:start+limit]
        tracks_list = sp.tracks(tracks)
        features = sp.audio_features(tracks=tracks)
        for index, track in enumerate(tracks_list['tracks']):

            try:
                if (start+index)%100==0: 
                    print(index+1+start)
                preview_urls_all.append(track['preview_url'])
                track_ids_all.append(track['id'])
                popularity.append(track['popularity'])
                names_all.append(track['name'])
                artists_all.append(track['artists'][0]['name'])
                track_numbers_all.append(track['track_number'])
                songData = features[index]
                aud_features[index+start][0] = songData['acousticness']
                aud_features[index+start][1] = songData['danceability']
                aud_features[index+start][2] = songData['energy']
                aud_features[index+start][3] = songData['instrumentalness']
                aud_features[index+start][4] = songData['key']
                aud_features[index+start][5] = songData['liveness']
                aud_features[index+start][6] = songData['loudness']
                aud_features[index+start][7] = songData['mode']
                aud_features[index+start][8] = songData['tempo']
                aud_features[index+start][9] = songData['time_signature']
                aud_features[index+start][10] = songData['valence']
                aud_features[index+start][11] = songData['duration_ms']
                aud_features[index+start][12] = songData['speechiness']
                 
            except:
                print("NO METADATA")


    track_data = pd.DataFrame({
    'preview_url': preview_urls_all,
    'id': track_ids_all,
    'popularity': popularity,
    'artists': artists_all,
    'names': names_all,
    'acousticness': aud_features[:,0],
    'danceability': aud_features[:,1],
    'energy': aud_features[:,2],
    'instrumentalness': aud_features[:,3],
    'key': aud_features[:,4],
    'liveness': aud_features[:,5],
    'loudness': aud_features[:,6],
    'mode': aud_features[:,7],
    'tempo': aud_features[:,8],
    'time_signature': aud_features[:,9],
    'valence': aud_features[:,10],
    'duration': aud_features[:,11],
    'speechiness': aud_features[:,12]
    
   
    })
    #q.put(track_data)

    #name = multiprocessing.current_process().name
    print('Exiting')
    return track_data


In [77]:
features_df = get_popularity(296,track_ids=track_ids)

Starting
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901
10001
10101
10201
10301
10401
10501
10601
10701
10801
10901
11001
11101
11201
11301
11401
11501
11601
11701
11801
11901
12001
12101
12201
12301
12401
12501
12601
12701
12801
12901
13001
13101
13201
13301
13401
13501
13601
13701
13801
13901
14001
14101
14201
14301
14401
14501
14601
14701
Exiting


### Examine resulting dataframe

In [78]:
features_df

,acousticness,artists,danceability,duration,energy,id,instrumentalness,key,liveness,loudness,mode,names,popularity,preview_url,speechiness,tempo,time_signature,valence
0,0.987000,Deep Divers,0.239,331843.0,0.01070,1tPGSiwtvmMVAqmniJ7oHp,0.916000,7.0,0.0912,-29.827,0.0,Sea Of Silence,39,https://p.scdn.co/mp3-preview/4b77c40b289080cc...,0.0315,100.031,4.0,0.1380
1,0.201000,Sheryl Crow,0.608,238733.0,0.93800,1tpShM80HMT9vKjRiA3EzW,0.004070,2.0,0.9660,-4.544,1.0,Merry Christmas Baby - Live,12,https://p.scdn.co/mp3-preview/052fb458e56162f4...,0.1980,96.064,4.0,0.7560
2,0.247000,RedOne,0.690,207720.0,0.70200,1TPUbLdzpfgF7wrPEXIAAB,0.000000,11.0,0.2620,-6.103,1.0,Don't You Need Somebody (feat. Enrique Iglesia...,64,https://p.scdn.co/mp3-preview/7e91506026ba3900...,0.0901,92.996,4.0,0.5920
3,0.861000,John Moreland,0.389,254619.0,0.20800,1tQGRq2WOBXjL3JWdWMONg,0.000008,2.0,0.1920,-11.669,1.0,You Don't Care for Me Enough to Cry,40,https://p.scdn.co/mp3-preview/c305d06bef43a04f...,0.0359,90.346,4.0,0.2270
4,0.037200,Placebo,0.387,254760.0,0.89800,1TqmZVIyeD4aN4Wa0wbnfy,0.000040,8.0,0.1100,-6.835,1.0,Pure Morning,54,https://p.scdn.co/mp3-preview/42be861caeeb76ac...,0.0675,81.354,4.0,0.1970
5,0.725000,Good Morning,0.645,240000.0,0.92200,1TquBahz02HCaxj8jy5tTo,0.877000,4.0,0.1840,-5.333,0.0,Cab Deg,44,https://p.scdn.co/mp3-preview/f3464ab234b07ec3...,0.0289,126.310,4.0,0.8310
6,0.062300,Talking Heads,0.646,259600.0,0.88000,1Tr4K5MU5XYE44umXGDndd,0.157000,2.0,0.3550,-7.753,1.0,Once In A Lifetime,37,https://p.scdn.co/mp3-preview/2fb67a5637d04587...,0.0637,116.916,4.0,0.8040
7,0.000073,Oasis,0.316,463120.0,0.87900,1tRlCCMVLSVQrrN9PPgtf0,0.000176,11.0,0.1030,-3.206,0.0,D'You Know What I Mean?,21,https://p.scdn.co/mp3-preview/721b11ba6bd69378...,0.0422,80.036,4.0,0.2350
8,0.000123,Jay Robinson,0.804,358200.0,0.91200,1TScJLsDWzDVBFpCU5vJDo,0.829000,3.0,0.0681,-7.863,0.0,Heiss,20,https://p.scdn.co/mp3-preview/71d988f627707a18...,0.0592,123.993,4.0,0.6000
9,0.872000,Aretha Franklin,0.161,200600.0,0.40900,1TSjS0q24cPW2S4pYj2wEK,0.000679,8.0,0.7050,-9.532,1.0,Nessun Dorma,26,https://p.scdn.co/mp3-preview/d23983c39b4653c2...,0.0335,80.348,4.0,0.1100


### Output resulting data frame to a csv

In [79]:
features_df.to_csv('song_features_laura.csv', encoding='utf-8')

In [46]:
features_df[features_df['duration']>500000]

,acousticness,artists,danceability,duration,energy,id,instrumentalness,key,liveness,loudness,mode,names,popularity,preview_url,tempo,time_signature,valence
34,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
42,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
74,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
82,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
114,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
122,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
143,0.6670,Tim Piggott Smith,0.6260,1048733.0,0.2280,1uKTddJWp8sTcSJ7mT6SjE,0.000000,7.0,0.3680,-23.905,0.0,"Dr Jeckyll & Mr Hyde: Chapter 5, Confrontation",7,https://p.scdn.co/mp3-preview/9ee4ec1d3ae8deb4...,82.618,4.0,0.4570
154,0.7010,Davidji,0.4800,855387.0,0.3370,1u4yUGqtLb7xFzl38qeW55,0.000015,2.0,0.0859,-15.569,1.0,Beginning Your Journey,16,https://p.scdn.co/mp3-preview/63dc091b107efa7a...,77.033,4.0,0.3840
162,0.9930,Thomas Tallis,0.0751,510333.0,0.0693,1Ud64pWX63TPOgTqoZlkIZ,0.098200,7.0,0.1110,-29.366,1.0,The Lamentations: First Set,26,https://p.scdn.co/mp3-preview/34bd23b19dd8bef5...,78.147,4.0,0.0382
183,0.6670,Tim Piggott Smith,0.6260,1048733.0,0.2280,1uKTddJWp8sTcSJ7mT6SjE,0.000000,7.0,0.3680,-23.905,0.0,"Dr Jeckyll & Mr Hyde: Chapter 5, Confrontation",7,https://p.scdn.co/mp3-preview/9ee4ec1d3ae8deb4...,82.618,4.0,0.4570


In [7]:
if __name__ == '__main__':
    q = Queue()
    for offset in [0,500,1000,1500,2000]:
    #for offset in [0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000,10500,11000,11500,12000,12500,13000,13500,14000,14500]:
        Process(target=get_popularity, args=(offset,10,q, track_ids)).start()
    #p.start()
    #print(q.get())    # prints "[42, None, 'hello']"
    #p.join()

Starting
Starting
Starting
0
0
Starting
Starting
0
0
0


In [ ]:
all_dfs = []
for _ in range(5):
    all_dfs.append(q.get())

In [32]:
playlist_data = all_dfs[0]
for each in all_dfs[1:]:
    playlist_data = playlist_data.append(each,ignore_index=True)
playlist_data.head()

,artists,id,names,popularity,preview_url
0,"[{u'name': u'Deep Divers', u'external_urls': {...",1tPGSiwtvmMVAqmniJ7oHp,Sea Of Silence,39,https://p.scdn.co/mp3-preview/4b77c40b289080cc...
1,"[{u'name': u'Sheryl Crow', u'external_urls': {...",1tpShM80HMT9vKjRiA3EzW,Merry Christmas Baby - Live,12,https://p.scdn.co/mp3-preview/052fb458e56162f4...
2,"[{u'name': u'RedOne', u'external_urls': {u'spo...",1TPUbLdzpfgF7wrPEXIAAB,Don't You Need Somebody (feat. Enrique Iglesia...,64,https://p.scdn.co/mp3-preview/7e91506026ba3900...
3,"[{u'name': u'John Moreland', u'external_urls':...",1tQGRq2WOBXjL3JWdWMONg,You Don't Care for Me Enough to Cry,40,https://p.scdn.co/mp3-preview/c305d06bef43a04f...
4,"[{u'name': u'Placebo', u'external_urls': {u'sp...",1TqmZVIyeD4aN4Wa0wbnfy,Pure Morning,54,https://p.scdn.co/mp3-preview/42be861caeeb76ac...


In [34]:
#playlist_data.to_csv("laura_data.csv")
#playlist_data['artists'][0][0]['name']
playlist_data.to_csv('all_data.csv', encoding='utf-8')

In [35]:
import glob, os
os.chdir("/Users/sonumehta/Desktop/Sem2/Capstone/PyAudio_tools/pyAudioAnalysis/tracks_total")

data_frame1 = pd.read_csv('all_data.csv')
data_frame1.head()


,Unnamed: 0,artists,id,names,popularity,preview_url
0,0,"[{u'name': u'Deep Divers', u'external_urls': {...",1tPGSiwtvmMVAqmniJ7oHp,Sea Of Silence,39,https://p.scdn.co/mp3-preview/4b77c40b289080cc...
1,1,"[{u'name': u'Sheryl Crow', u'external_urls': {...",1tpShM80HMT9vKjRiA3EzW,Merry Christmas Baby - Live,12,https://p.scdn.co/mp3-preview/052fb458e56162f4...
2,2,"[{u'name': u'RedOne', u'external_urls': {u'spo...",1TPUbLdzpfgF7wrPEXIAAB,Don't You Need Somebody (feat. Enrique Iglesia...,64,https://p.scdn.co/mp3-preview/7e91506026ba3900...
3,3,"[{u'name': u'John Moreland', u'external_urls':...",1tQGRq2WOBXjL3JWdWMONg,You Don't Care for Me Enough to Cry,40,https://p.scdn.co/mp3-preview/c305d06bef43a04f...
4,4,"[{u'name': u'Placebo', u'external_urls': {u'sp...",1TqmZVIyeD4aN4Wa0wbnfy,Pure Morning,54,https://p.scdn.co/mp3-preview/42be861caeeb76ac...


In [36]:
features = []
track_final = []
popularity_final = []
for i in range(data_frame1['id'].shape[0]):
    id1  =data_frame1['id'].iloc[i]
    name = "track"+ id1+".wav_st.csv"

    a  = pd.read_csv(name, header = None)
    if (np.ravel(a).shape[0]) == 1020:
        
        b = np.ravel(np.reshape(np.ravel(a), (1020,1)))
    

        features.append(b)
        track_final.append(id1)
        popularity_final.append(data_frame1['popularity'].iloc[i])
features = np.array(features)